### Tests for compiler and knowledge base

In [36]:
from openagent import compiler
from openagent.llms._openai import OpenAI
from openagent.vectorstores.chroma import Chroma
from openagent.text_splitter import CharacterTextSplitter
from openagent.knowledgebase.document_loaders import TextLoader
from openagent.vectorstores.embeddings.openai import OpenAIEmbeddings

# load .env file in format :
# OPENAI_API_KEY=sk-xxx
from dotenv import load_dotenv
load_dotenv()


llmcc = OpenAI(
    model="gpt-3.5-turbo"
)

llmtc = OpenAI(
    model="text-davinci-003"
)

#### Test case for known data (already in llm context)

In [ ]:
pg_data = r'test_data\paul_graham_essay.txt'
loader = TextLoader(pg_data)
docs = loader.load()

splitter = CharacterTextSplitter(chunk_overlap=400, chunk_size=2000)
splitted = splitter.split_documents(docs)

openai_embeddings = OpenAIEmbeddings()
chroma = Chroma(embedding_function=openai_embeddings)

chroma.add_documents(splitted)

In [38]:
def knowledgebase(query):
    doc = chroma.similarity_search(query=query, top_k=1)
    return doc[0][0].page_content

In [39]:
knowledgebase('we had enough money')

"Another thing I didn't get at the time is that growth rate is the ultimate test of a startup. Our growth rate was fine. We had about 70 stores at the end of 1996 and about 500 at the end of 1997. I mistakenly thought the thing that mattered was the absolute number of users. And that is the thing that matters in the sense that that's how much money you're making, and if you're not making enough, you might go out of business. But in the long term the growth rate takes care of the absolute number. If we'd been a startup I was advising at Y Combinator, I would have said: Stop being so stressed out, because you're doing fine. You're growing 7x a year. Just don't hire too many more people and you'll soon be profitable, and then you'll control your own destiny.\n\nAlas I hired lots more people, partly because our investors wanted me to, and partly because that's what startups did during the Internet Bubble. A company with just a handful of employees would have seemed amateurish. So we didn't

In [40]:
# chatbot demonstration

def chatbot(input, chat_loop):

    knowledge = knowledgebase(input)
    if chat_loop is None:
        chat_loop = compiler(
                    '''{{#system}}You are a helpful assistant. {{/system}}

                        {{~#geneach 'conversation' stop=False}}

                        {{#user~}}
                        Query: {{set 'this.user_text' (await 'user_text') hidden=False}}
                        Use this below information for your answer. Do not talk about the information provided, Just answer the query as an intelligent Assistant
                        {{knowledge}}
                        {{~/user}}

                        {{#assistant~}}
                        {{gen 'this.response' temperature=0 max_tokens=300}}
                        {{~/assistant}}
                        
                        {{~/geneach}}''', llm = llmcc, silent = True, caching = False) 
    
    chat_loop = chat_loop(user_text = input, knowledge = knowledge) 
    return chat_loop['conversation'][-2]['response'], chat_loop

In [41]:
resp, loop = chatbot("If I don't hire too many more people and will I become profitable?", None)
print(resp)

Based on the information provided, if you don't hire too many more people, you have a good chance of becoming profitable. The growth rate of your startup is strong, growing 7x a year, which indicates positive potential for financial success. However, hiring too many additional employees may hinder your profitability. It is important to control your expenses and focus on sustainable growth to ensure long-term success and control over your company's destiny.


In [42]:
resp, loop = chatbot("How to get registered in YC?", loop)
print(resp)

To get registered in Y Combinator (YC), you can apply for their batch program. YC operates on a batch model, where they fund a group of startups twice a year and provide intensive support for three months. They initially started the program to gain experience as investors and provide a unique opportunity for founders.

To apply, keep an eye out for their application period and follow the instructions provided on their website. Typically, they require applicants to submit essays or applications that showcase their startup idea, team, and potential. YC values diverse backgrounds and encourages applications from both undergraduates and recent graduates.

By applying to YC, you have the chance to be part of a program that offers mentorship, resources, and networking opportunities to help your startup succeed.


#### Test case for hypothetical data (not in llm context)

In [ ]:
pg_data = r'test_data\meteoric\the_meteoric_guardian.txt'
loader = TextLoader(pg_data)
docs = loader.load()

splitter = CharacterTextSplitter(chunk_overlap=20, chunk_size=1024)
splitted = splitter.split_documents(docs)

openai_embeddings = OpenAIEmbeddings()
chroma = Chroma(embedding_function=openai_embeddings)

chroma.add_documents(splitted)

In [78]:
chroma.similarity_search(query="What did Alex said to Xarnak in their first meeting?", top_k=1)

[(Document(page_content="ALEX:\n(Defiant)\nI am no longer the scared child you remember, Xarnak. I am the protector of this planet, and I won't let you harm it.\n\n(The two adversaries lock eyes, their powers crackling around them.)\n\nNARRATOR (V.O.):\nThe fate of Earth hung in the balance as Alex and Xarnak prepared to engage in an epic battle that would decide the planet's destiny.\n\n(With a mighty roar, Xarnak unleashes a massive energy blast toward Alex. In a display of agility, Alex dodges the attack with astonishing precision.)\n\nALEX:\n(Focused)\nYou can't defeat me, Xarnak. I've become more than you ever imagined.\n\n(Xarnak's face contorts with rage, and he charges at Alex, unleashing a barrage of powerful strikes. But Alex's speed proves too much for the alien warlord, who struggles to keep up.)\n\nNARRATOR (V.O.):\nThe Meteoric Guardian's super-speed gave him the upper hand in the battle, as he evaded Xarnak's attacks and countered with remarkable accuracy.", metadata={'s

In [79]:
def knowledgebase(query):
    doc = chroma.similarity_search(query=query, top_k=1)
    return doc[0][0].page_content

In [80]:
# chatbot demonstration

def chatbot(input, chat_loop):

    knowledge = knowledgebase(input)
    if chat_loop is None:
        chat_loop = compiler(
                    '''{{#system}}You are a helpful assistant. {{/system}}

                        {{~#geneach 'conversation' stop=False}}

                        {{#user~}}
                        Query: {{set 'this.user_text' (await 'user_text') hidden=False}}
                        Use this below information for your answer. Do not talk about the information provided, Just answer the query as an intelligent Assistant
                        {{knowledge}}
                        {{~/user}}

                        {{#assistant~}}
                        {{gen 'this.response' temperature=0 max_tokens=300}}
                        {{~/assistant}}
                        
                        {{~/geneach}}''', llm = llmcc, silent = True, caching = False) 
    
    chat_loop = chat_loop(user_text = input, knowledge = knowledge) 
    return chat_loop['conversation'][-2]['response'], chat_loop

In [81]:
resp, loop = chatbot("What did Alex said to Xarnak in their first meeting?", None)
print(resp)

I am no longer the scared child you remember, Xarnak. I am the protector of this planet, and I won't let you harm it. You can't defeat me, Xarnak. I've become more than you ever imagined.


#### Using llama-index

In [53]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader('test_data\meteoric').load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("What did Alex said to Xarnak in their first meeting?")
print(response)


ALEX: "I am no longer the scared child you remember, Xarnak. I am the protector of this planet, and I won't let you harm it."
